In [ ]:
#gerekli kütüphane
!pip install opencv-contrib-python --force-reinstall

In [ ]:
#kendi driveımıza girelim
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#yolov4 dosyasının içindekileri liste şeklinde yükleyelim
!ls /content/drive/MyDrive/yolov4

In [ ]:
#githubdan darkneti klonlayalım
!git clone https://github.com/AlexeyAB/darknet.git /mydrive/Yolov4/darknet

In [ ]:
#yolov4 dosyasındaki darkneti değiştirme
%cd /mydrive/Yolov4/darknet/

In [ ]:
#darknetin içindekileri listeleyelim
!ls /mydrive/Yolov4/darknet

In [7]:
# darknetimiz düzgün çalışması için değiştirilmesi gereken fonksiyonların değerlerini değiştirelim
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
#dosyalarımzı derleyelim bağlayalım
!make

In [ ]:
#dosyaların bazılarının içini düzenleme işlemi
%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..
%rm -rf cfg/
%mkdir cfg

In [ ]:
#zip dosyasını açarak data klasörüne ekleyelim
!unzip /content/drive/MyDrive/yolov4/egitim_data/spot_images11.zip -d data/

In [11]:
#cfg dosyasını darknet içindeki cfg klasörüne kopyalayalım
!cp /content/drive/MyDrive/darknet/yolov4_egitim.cfg cfg

In [12]:
# ağırlık ve sınıf dosyalarımızı data klasörümüzün içine kopyalayalım
!cp /content/drive/MyDrive/yolov4/egitim_data/ders.data data
!cp /content/drive/MyDrive/yolov4/egitim_data/egitim_data.names  data

In [13]:
# txt dosyalarımızı darknet içindeki data klasörüne kopyalayalım
!cp /content/drive/MyDrive/darknet/egitim_data/spot_testing.txt data
!cp /content/drive/MyDrive/darknet/egitim_data/spot_training.txt data

In [ ]:
#data dizinindeki içindeki klasörleri listeleyelim
!ls data

In [ ]:
#txt içeriğini listeleyelim
!cat data/spot_testing.txt

In [ ]:
# yolov4 dosyasını indirelim
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
!./darknet detector train /content/drive/MyDrive/yolov4/egitim_data/ders.data cfg/yolov4_egitim.cfg yolov4.conv.137 -dont_show -map

In [18]:
# gerekli kütüphanelerin yüklenmesi
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import os
import time
import html
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:

# JavaScript objesini bir OpenCV görüntüsüne dönüştürme işlemi
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [20]:
# Web kamerasını giriş olarak kullanarak canlı video akışını düzgün bir şekilde oluşturmak için JavaScript kodumuz
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
# Web kamerasından video yayınına başlama işlemi
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0

# load our YOLO object detector trained on our custom dataset dataset (02 classes)
net = cv2.dnn.readNetFromDarknet('cfg/yolov4_egitim.cfg', '/content/drive/MyDrive/darknet/yolov4_egitim_final.weights')
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

DEFAULT_CONFIANCE = 0.5
THRESHOLD = 0.4
# load the COCO class labels our YOLO model was trained on
with open('data/egitim_data.names', 'r') as f:
    LABELS = f.read().splitlines()

# initialize the video stream, pointer to output video file
#cap = cv2.VideoCapture(0)

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    image = js_to_image(js_reply["img"])

    #_,image=cap.read()
    height, width, _ = image.shape

    blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    # initialize our lists of detected bounding boxes, confidences, and class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > DEFAULT_CONFIANCE:
                # scale the bounding box coordinates back relative to
                # the size of the image, keeping in mind that YOLO
                # actually returns the center (x, y)-coordinates of
                # the bounding box followed by the boxes' width and
                # height
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, W, H) = box.astype("int")
                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (W / 2))
                y = int(centerY - (H / 2))
                # update our list of bounding box coordinates,
                # confidences, and class IDs
                boxes.append([x, y, int(W), int(H)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, DEFAULT_CONFIANCE, THRESHOLD)

    # initialize a list of colors to represent each possible class label
    COLORS = np.random.uniform(0,255,size=(len(boxes), 3))

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # ensure at least one detection exists
    if len(indexes) > 0:
        # loop over the indexes we are keeping
        for i in indexes.flatten():
            # extract the bounding box coordinates
            (x, y, w, h) = boxes[i]
            # draw a bounding box rectangle and label on the frame
            color = COLORS[i]
            text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            bbox_array = cv2.rectangle(bbox_array, (x, y), (x + w, y + h), color, 2)
            bbox_array = cv2.putText(bbox_array, text, (x, y + 20 ), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)

    #cv2.imshow('Image', image)
    #if cv2.waitKey(1)==ord('q'):
     #   break

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes